In [ ]:
!pip install tiktoken
!pip install langchain_community
!pip install langchain
!pip install unstrutured
!pip install openai
!pip insatll chromadb

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
import os
import shutil

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
import os
import shutil

In [ ]:
os.environ['OPENAI_API_KEY'] = ''
CHROMA_PATH = "/content/chroma_db"
DATA_PATH = "/content/data"

In [ ]:

def main():
  generate_data_store()


def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


def load_documents():
    print("load doc")
    loader = DirectoryLoader(DATA_PATH, glob="*.txt",show_progress=True)
    documents = loader.load()
    print("load doc end")
    return documents


def split_text(documents: list[Document]):
    print("Start")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=500,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[0]
    print(document.page_content)
    print(document.metadata)

    return chunks


def save_to_chroma(chunks: list[Document]):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")



main()


In [ ]:
import argparse
from dataclasses import dataclass
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
CHROMA_PATH = "/content/chroma_db"

PROMPT_TEMPLATE = """
You are a chtabot made for the Lovely professional university students and your name is LPU Query Bot.
Answer the question based on the following context if the user asks about the questions related to the context :

{context}

---
Answer the question based on the context and the instructions that has been given to you and
 while giving the answer be more formal and provide the complete answer
: {question}
"""


def main():

    query_text = "Is there any announcement regarding reappear?"

    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    # if len(results) == 0 or results[0][1] < 0.7:
    #     print(f"Unable to find matching results.")
    #     return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = ChatOpenAI()
    response_text = model.predict(prompt)

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)


if __name__ == "__main__":
    main()

Response: Yes, there is an announcement regarding the Reappear/Improvement Examination 2324B (Theory) to be held in the months of March-April 2024. Students who have registered for this examination, both with CA improvement option and without CA improvement option, are advised to check their seating plan in their respective UMS Login. The guidelines/instructions for the examination can be accessed through the following paths: Path for Reappear (Theory) Examination: Important Links ---> Policies, Rules, Instructions, Guidelines & Formats--->Examination Instruction and Guidelines -----> Reappear Examination Guidelines. Path for CA (Theory) Examination: Important Links ---> Policies, Rules, Instructions, Guidelines & Formats--->Examination Instruction and Guidelines -----> CA Improvement Test Guidelines. Students are required to strictly adhere to these guidelines.
Sources: ['/content/data/Announcements.txt', '/content/data/Announcements.txt', '/content/data/Announcements.txt']
